In [1]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import pandas as pd
pd.set_option('display.max_colwidth', None)

#import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [2]:
examples = [
    'My wife told me to stop impersonating a flamingo. I had to put my foot down.',
    'I went to buy some camo pants but couldn’t find any.',
    'I failed math so many times at school, I can’t even count.',
    'Did you know that the boomerang is Australias biggest export? Its also their biggest import.',
    'Don’t you hate it when someone answers their own questions? I do.',
    'Germany is the second most populous country in Europe after Russia.',
    "Cost of living calculations are also used to compare the cost of maintaining a certain standard of living in different geographic areas.",
    "Along with Hong Kong, South Korea, and Taiwan, Singapore is one of the Four Asian Tigers, and has surpassed its peers in terms of Gross Domestic Product (GDP) per capita.",
    "Nowadays Italy is the fifth most visited country in international tourism.",
    "This TF Hub model uses the implementation of BERT from the TensorFlow Models repository on GitHub."
]

df = pd.DataFrame({"Sentence": examples, "isJoke": ["yes","yes","yes","yes","yes","no","no","no","no","no"]})

In [3]:
directories_in_curdir = sorted(list(filter(os.path.isdir, os.listdir(os.curdir))))
available_models = [i for i in directories_in_curdir if i.startswith('model_')]

for model in available_models:
    
    reloaded_model = tf.saved_model.load(model)

    def is_funny(inputs, results):
      result_for_printing = \
        [f'{inputs[i]:<30}\n FUNNY SCORE: {results[i][0]:.6f}\n'
                             for i in range(len(inputs))]
      #print(*result_for_printing, sep='\n')
      print("Evaluating "+model)


    examples = [
        'My wife told me to stop impersonating a flamingo. I had to put my foot down.',
        'I went to buy some camo pants but couldn’t find any.',
        'I failed math so many times at school, I can’t even count.',
        'Did you know that the boomerang is Australias biggest export? Its also their biggest import.',
        'Don’t you hate it when someone answers their own questions? I do.',
        'Germany is the second most populous country in Europe after Russia.',
        "Cost of living calculations are also used to compare the cost of maintaining a certain standard of living in different geographic areas.",
        "Along with Hong Kong, South Korea, and Taiwan, Singapore is one of the Four Asian Tigers, and has surpassed its peers in terms of Gross Domestic Product (GDP) per capita.",
        "Nowadays Italy is the fifth most visited country in international tourism.",
        "This TF Hub model uses the implementation of BERT from the TensorFlow Models repository on GitHub."
    ]

    reloaded_results = tf.sigmoid(reloaded_model(tf.constant(examples)))

    is_funny(examples, reloaded_results)
    
    df[model] = reloaded_results
    
df

Evaluating model_albert_en_base_20221121-143742
Evaluating model_small_bert_bert_en_uncased_L-2_H-128_A-2_20221121-091025
Evaluating model_small_bert_bert_en_uncased_L-2_H-128_A-2_20221121-135444
Evaluating model_small_bert_bert_en_uncased_L-4_H-128_A-2_20221121-191719
Evaluating model_small_bert_bert_en_uncased_L-4_H-128_A-2_20221121-223841
Evaluating model_small_bert_bert_en_uncased_L-4_H-512_A-8_20221120-232512


,Sentence,isJoke,model_albert_en_base_20221121-143742,model_small_bert_bert_en_uncased_L-2_H-128_A-2_20221121-091025,model_small_bert_bert_en_uncased_L-2_H-128_A-2_20221121-135444,model_small_bert_bert_en_uncased_L-4_H-128_A-2_20221121-191719,model_small_bert_bert_en_uncased_L-4_H-128_A-2_20221121-223841,model_small_bert_bert_en_uncased_L-4_H-512_A-8_20221120-232512
0,My wife told me to stop impersonating a flamingo. I had to put my foot down.,yes,0.994432,0.981821,0.987288,0.997642,0.995972,0.999950
1,I went to buy some camo pants but couldn’t find any.,yes,0.993870,0.980774,0.989220,0.373588,0.973850,0.000080
2,"I failed math so many times at school, I can’t even count.",yes,0.978977,0.971235,0.970713,0.684414,0.973444,0.001421
3,Did you know that the boomerang is Australias biggest export? Its also their biggest import.,yes,0.989559,0.982150,0.979083,0.997054,0.994753,0.999976
4,Don’t you hate it when someone answers their own questions? I do.,yes,0.989809,0.985276,0.985770,0.442658,0.991864,0.000124
5,Germany is the second most populous country in Europe after Russia.,no,0.100006,0.177129,0.052697,0.956888,0.719344,0.999804
6,Cost of living calculations are also used to compare the cost of maintaining a certain standard of living in different geographic areas.,no,0.714653,0.468616,0.557338,0.987005,0.902646,0.999966
7,"Along with Hong Kong, South Korea, and Taiwan, Singapore is one of the Four Asian Tigers, and has surpassed its peers in terms of Gross Domestic Product (GDP) per capita.",no,0.071859,0.072069,0.044583,0.618736,0.081986,0.999388
8,Nowadays Italy is the fifth most visited country in international tourism.,no,0.201814,0.174889,0.095321,0.946334,0.807160,0.999803
9,This TF Hub model uses the implementation of BERT from the TensorFlow Models repository on GitHub.,no,0.492181,0.197828,0.477216,0.913455,0.471453,0.999863


In [4]:
df1 = pd.DataFrame({"Models": available_models})

for model in available_models:
    df1.loc[df1["Models"] == model,"Quality KPI (closer to 5 the best)"] = df[model].sum()
    df1['Delta to 5'] = df1['Quality KPI (closer to 5 the best)'] - 5

df1.sort_values(by='Delta to 5', ascending=True, inplace=True)
df1 = df1.reset_index(drop=True)
df1

,Models,Quality KPI (closer to 5 the best),Delta to 5
0,model_small_bert_bert_en_uncased_L-2_H-128_A-2_20221121-091025,5.991787,0.991787
1,model_small_bert_bert_en_uncased_L-2_H-128_A-2_20221121-135444,6.139230,1.139230
2,model_albert_en_base_20221121-143742,6.527159,1.527159
3,model_small_bert_bert_en_uncased_L-4_H-512_A-8_20221120-232512,7.000374,2.000374
4,model_small_bert_bert_en_uncased_L-4_H-128_A-2_20221121-223841,7.912474,2.912474
5,model_small_bert_bert_en_uncased_L-4_H-128_A-2_20221121-191719,7.917773,2.917773
